In [90]:
import pandas as pd
import numpy as np
import os
from datetime import datetime,date,timedelta,timezone
import calendar
import json


from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud.exceptions import NotFound
from google.api_core.exceptions import BadRequest



In [91]:
collectionDate='2023-07-09 00:10' # comment
# uncomment and indent
# import functions_framework
# @functions_framework.http
# def collect_prediction_result(request):   # run on clound function

# def collect_prediction_result(collectionDate): # migrate


# Init parameter

In [92]:
model_id="Incident_60To5_E150S15B32-M0122-0723"

mode=1

if mode==1: # Migrate to backfill data and Test 
    logDate=collectionDate
    log_date=datetime.strptime(logDate,'%Y-%m-%d %H:%M')
    log_timestamp=datetime.strptime(logDate,'%Y-%m-%d %H:%M')
else: # On weekly basis
    log_timestamp=datetime.now(timezone.utc)
    log_date=datetime.strptime(log_timestamp.strftime('%Y-%m-%d'),'%Y-%m-%d')

week_day=log_date.weekday()
day_name=calendar.day_name[log_date.weekday()]

print(f"Date to collect data on {log_date.strftime('%Y-%m-%d')} {day_name}(Idx:{week_day}) at {log_timestamp}")

if  week_day!=6:
    raise Exception("Monday is allowed  as Collection Date for forcasting result.")   

print(f"week_day={week_day}")

genTableSchema=False
metric_name='mae'


date_col='date'
genTableSchema=True
metric_name='mae'


Date to collect data on 2023-07-09 Sunday(Idx:6) at 2023-07-09 00:10:00
week_day=6


# BigQuery Setting & Configuration Variable

In [93]:

projectId='smart-data-ml'
dataset_id='SMartDW'

table_data_id=f"{projectId}.{dataset_id}.daily_incident"
table_id = f"{projectId}.{dataset_id}.prediction_daily_incident"
table_perf_id= f"{projectId}.{dataset_id}.model_perfromance_daily_incident"

print(table_id)
print(table_data_id)
print(table_perf_id)

# client = bigquery.Client(project=projectId )
credentials = service_account.Credentials.from_service_account_file(r'C:\Windows\smart-data-ml-91b6f6204773.json')
client = bigquery.Client(credentials=credentials, project=projectId)

def load_data_bq(sql:str):
    query_result=client.query(sql)
    df=query_result.to_dataframe()
    return df

smart-data-ml.SMartDW.prediction_daily_incident
smart-data-ml.SMartDW.daily_incident
smart-data-ml.SMartDW.model_perfromance_daily_incident


# Create Start to End Date By Getting Last Date of Week

In [94]:
# get  prev prediction  from  get end prediction to beginneg or predicton of week 
endX=log_date
days_in_weeks=7
startX=endX+timedelta(days=-(days_in_weeks-1))#
print(f"Collection data the last  {(endX-startX).days+1} days :From {startX.strftime('%A %d-%m-%Y')} To {endX.strftime('%A %d-%m-%Y')}")

endX=endX.strftime('%Y-%m-%d')
startX=startX.strftime('%Y-%m-%d')

print(f"Convert start and end data {startX} - {endX} to string")

Collection data the last  7 days :From Monday 03-07-2023 To Sunday 09-07-2023
Convert start and end data 2023-07-03 - 2023-07-09 to string


# Check where the given date collected data or not?

In [95]:
sqlCheck=f"""
select collection_timestamp from `{table_perf_id}`
where date(collection_timestamp)='{log_date.strftime('%Y-%m-%d')}' and model_id='{model_id}'
"""

print(sqlCheck)
dfCheckDate=load_data_bq(sqlCheck)
if  dfCheckDate.empty==False:
    print(f"Collection data on {log_date} for {model_id} found, no any action")
    # uncomment
    #return f"Collection data on {log_date} for {model_id} found, no any action"
else:
    print(f"We are ready to Collect data on {log_date}")


select collection_timestamp from `smart-data-ml.SMartDW.model_perfromance_daily_incident`
where date(collection_timestamp)='2023-07-09' and model_id='Incident_60To5_E150S15B32-M0122-0723'

We are ready to Collect data on 2023-07-09 00:10:00


# Retrive forecasting result data to Dictionary

In [97]:
def get_forecasting_result_data(request):

    if   request is not None:  
        start_date=request["start_date"]
        end_date=request["end_date"]
        model_id=request["model_id"]
    else:
        raise Exception("No request parameters such as start_date,end_date")

    
    print("1.How far in advance does model want to  make prediction")
    sqlOutput=f"""
    select t.pred_timestamp,t.prediction_date,t_pred.date,t_pred.count_incident
    from  `{table_id}` t cross join unnest(t.prediction_result) t_pred
    where (t.prediction_date>='{start_date}' and  t.prediction_date<='{end_date}')
    and t.model_id='{model_id}' and t_pred.type='prediction'
    order by  t.pred_timestamp,t.prediction_date,t_pred.date
    """
    print(sqlOutput)
    dfOutput=load_data_bq(sqlOutput)
    dfOutput[date_col]=pd.to_datetime(dfOutput[date_col],format='%Y-%m-%d')
    dfOutput.set_index(date_col,inplace=True)

    output_sequence_length=len(dfOutput)
    print(f"output_sequence_length={output_sequence_length}")
    

    print(dfOutput.info())
    print(dfOutput)
    print("================================================================================================")

    
    #get actual data since the fist day of input and the last day of output(if covered)
    startFinData=dfOutput.index.min().strftime('%Y-%m-%d')
    endFindData=dfOutput.index.max().strftime('%Y-%m-%d')
    print(f"2.Get Real Data  to compare to prediction from {startFinData} to {endFindData}")

    sqlData=f"""
    select {date_col},count_incident, datetime_imported, from `{table_data_id}` 
    where ({date_col}>='{startFinData}' and {date_col}<='{endFindData}')
    order by datetime_imported,{date_col}
    """
    
    print(sqlData)

    dfRealData=load_data_bq(sqlData)
    dfRealData=dfRealData.drop_duplicates(subset=[date_col],keep='last',)
    dfRealData[date_col]=pd.to_datetime(dfRealData[date_col],format='%Y-%m-%d')
    dfRealData.set_index(date_col,inplace=True)
    
    print(dfRealData.info())
    print(dfRealData)
    print("================================================================================================")

    return {'actual_no_incident':dfRealData,'predicted_no_incident':dfOutput }


print(f"================Get data from {startX}====to==={endX}================")
request={'start_date':startX,'end_date':endX,'model_id':model_id}
data=get_forecasting_result_data(request)
print(f"=======================================================================")

================Get data from 2023-07-03====to===2023-07-09================
1.How far in advance does model want to  make prediction

    select t.pred_timestamp,t.prediction_date,t_pred.date,t_pred.count_incident
    from  `smart-data-ml.SMartDW.prediction_daily_incident` t cross join unnest(t.prediction_result) t_pred
    where (t.prediction_date>='2023-07-03' and  t.prediction_date<='2023-07-09')
    and t.model_id='Incident_60To5_E150S15B32-M0122-0723' and t_pred.type='prediction'
    order by  t.pred_timestamp,t.prediction_date,t_pred.date
    
output_sequence_length=10
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 10 entries, 2023-07-09 to 2023-07-14
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype              
---  ------           --------------  -----              
 0   pred_timestamp   10 non-null     datetime64[ns, UTC]
 1   prediction_date  10 non-null     dbdate             
 2   count_incident   10 non-null     float64            
dtypes

# Create Predictive and Actual Value dataframe

In [98]:
print("List all trading day in the week")
myTradingDataList=data['predicted_no_incident']['prediction_date'].unique()
print(myTradingDataList)

List all trading day in the week
<DateArray>
[datetime.date(2023, 7, 8), datetime.date(2023, 7, 9)]
Length: 2, dtype: dbdate


In [99]:
print(f"========================dfX :Actual Price========================")
dfX=data['actual_no_incident'][['count_incident']]
dfX.columns=[f'actual_value']
print(dfX.info())
print(dfX)

========================dfX :Actual Price========================
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 6 entries, 2023-07-09 to 2023-07-14
Data columns (total 1 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   actual_value  6 non-null      float64
dtypes: float64(1)
memory usage: 96.0 bytes
None
            actual_value
date                    
2023-07-09           1.0
2023-07-10           6.0
2023-07-11           3.0
2023-07-12           2.0
2023-07-13           3.0
2023-07-14           2.0


In [100]:
dfAllForecastResult=pd.DataFrame(columns=['date','pred_value','actual_value','prediction_date'])
# actually , we can jon without spilting data by prediction_dtate
for date in  myTradingDataList: # trading day on giver week
    print(f"=========================dfPred:Predicted Price at {date}=========================")
    dfPred=data['predicted_no_incident'].query("prediction_date==@date")[['count_incident']]
    dfPred.columns=[f'pred_value']
    # print(dfPred.info())

    print("=====================dfCompare:Join Actual price to Predicted Price=================")
    dfCompare=pd.merge(left=dfPred,right=dfX,how='inner',right_index=True,left_index=True)
    dfCompare.reset_index(inplace=True)   
    dfCompare['prediction_date']=date.strftime('%Y-%m-%d')      
    print(dfCompare) 
    print(dfCompare.info())

    if len(dfCompare)>0 : # it will be join if there is at least one record to show actual vs pred
        dfAllForecastResult= pd.concat([dfAllForecastResult,dfCompare],ignore_index=True)
        print(f"=========================Appended Data Joined=========================")
    else:
        print("No Appendind Data due to no at least one record to show actual vs pred")

=========================dfPred:Predicted Price at 2023-07-08=========================
=====================dfCompare:Join Actual price to Predicted Price=================
        date  pred_value  actual_value prediction_date
0 2023-07-09         1.0           1.0      2023-07-08
1 2023-07-10         6.0           6.0      2023-07-08
2 2023-07-11         6.0           3.0      2023-07-08
3 2023-07-12         6.0           2.0      2023-07-08
4 2023-07-13         5.0           3.0      2023-07-08
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   date             5 non-null      datetime64[ns]
 1   pred_value       5 non-null      float64       
 2   actual_value     5 non-null      float64       
 3   prediction_date  5 non-null      object        
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 288.0+ bytes
None

In [101]:
print("========================dfAllForecastResult: All Predicton Result========================")
dfAllForecastResult[date_col]=dfAllForecastResult[date_col].dt.strftime('%Y-%m-%d')
print(dfAllForecastResult.info())
print(dfAllForecastResult)

========================dfAllForecastResult: All Predicton Result========================
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   date             10 non-null     object 
 1   pred_value       10 non-null     float64
 2   actual_value     10 non-null     float64
 3   prediction_date  10 non-null     object 
dtypes: float64(2), object(2)
memory usage: 448.0+ bytes
None
         date  pred_value  actual_value prediction_date
0  2023-07-09         1.0           1.0      2023-07-08
1  2023-07-10         6.0           6.0      2023-07-08
2  2023-07-11         6.0           3.0      2023-07-08
3  2023-07-12         6.0           2.0      2023-07-08
4  2023-07-13         5.0           3.0      2023-07-08
5  2023-07-10         6.0           6.0      2023-07-09
6  2023-07-11         6.0           3.0      2023-07-09
7  2023-07-12         5

# Calculate Metric

## Get sum distance between pred and actul value from prev rows

In [102]:
sqlMetric=f"""
with pred_actual_by_model as  
(
SELECT  detail.actual_value,detail.pred_value
from `{table_perf_id}`  t
 cross join unnest(t.pred_actual_data) as detail
 where t.model_id='{model_id}' and t.collection_timestamp<'{log_timestamp}'
)
select COALESCE( sum(abs(x.actual_value-x.pred_value)),0) as pred_diff_actual,count(*) as no_row  from pred_actual_by_model  x


"""

if genTableSchema==False:
    print(sqlMetric)

    dfMetric=load_data_bq(sqlMetric)
    prevSum=dfMetric.iloc[0,0]
    prevCount=dfMetric.iloc[0,1]

else:  # it is used if there are something changed in table schema
# for generating table schema
    prevSum=0
    prevCount=0

print(f"Prev Sum={prevSum} and Count={prevCount}")

Prev Sum=0 and Count=0


## Cal sum distance between pred and actul value from last rows

In [103]:
dfAllForecastResult['pred_diff_actual']=dfAllForecastResult.apply(lambda x : abs(x['pred_value']-x['actual_value']),axis=1)
recentSum=dfAllForecastResult['pred_diff_actual'].sum()
recentCount=len(dfAllForecastResult)

dfAllForecastResult=dfAllForecastResult.drop(columns=['pred_diff_actual'])
print(f"Recent Sum={recentSum} and Count={recentCount}")

#https://en.wikipedia.org/wiki/Mean_absolute_error
metric_value= round((prevSum+recentSum)/(prevCount+recentCount),2)
print(f"{metric_name} = {metric_value}")

Recent Sum=21.0 and Count=10
mae = 2.1


# Create Collection Performance Info Dataframe and Store 


In [104]:
masterDF=pd.DataFrame(data=[ [log_date,model_id,metric_name,metric_value,log_timestamp] ],
                columns=["collection_date","model_id","metric_name","metric_value","collection_timestamp"])

masterDF["collection_date"]=masterDF["collection_date"].dt.strftime('%Y-%m-%d') # for json format
masterDF["collection_timestamp"]=masterDF["collection_timestamp"].dt.strftime('%Y-%m-%d %H:%M:%S') # for json format
print(masterDF.info())
masterDF

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   collection_date       1 non-null      object 
 1   model_id              1 non-null      object 
 2   metric_name           1 non-null      object 
 3   metric_value          1 non-null      float64
 4   collection_timestamp  1 non-null      object 
dtypes: float64(1), object(4)
memory usage: 168.0+ bytes
None


,collection_date,model_id,metric_name,metric_value,collection_timestamp
0,2023-07-09,Incident_60To5_E150S15B32-M0122-0723,mae,2.1,2023-07-09 00:10:00


# Create Dataframe to  Json Data 

In [105]:
master_perf = json.loads(masterDF.to_json(orient = 'records')) # 1 main dataframe has 1 records
for  master in master_perf:
    detail= json.loads(dfAllForecastResult.to_json(orient = 'records'))
    master["pred_actual_data"]=detail

    
with open("no_incident_forecast_performance.json", "w") as outfile:
    json.dump( master_perf, outfile)

# Ingest Data to BigQuery

## Try to ingest data to get correct schema and copy the schema to create table including partion/cluster manually

In [106]:
try:
    table=client.get_table(table_perf_id)
    print("Table {} already exists.".format(table_id))
    
    job_config = bigquery.LoadJobConfig()
    job_config.source_format = bigquery.SourceFormat.NEWLINE_DELIMITED_JSON
    # Try to ingest data to get correct schema and copy the schema to create table including partiion/cluster manually
    job_config.write_disposition = bigquery.WriteDisposition.WRITE_APPEND 
    job = client.load_table_from_json(master_perf,table_perf_id, job_config = job_config)
    if job.errors is not None:
        print(job.error_result)
        print(job.errors)
        # uncomment
        # return "Error to load data to BigQuery"
    else:
        print(f"Import to bigquery successfully  {len(master_perf)} records")
        # print(table.schema)
except Exception as ex :
    print(str(ex))
    

    
#job_config.schema
# truncate table`pongthorn.FinAssetForecast.model_forecast_performance` 

Table smart-data-ml.SMartDW.prediction_daily_incident already exists.
Import to bigquery successfully  1 records


In [27]:
# uncomment
#return 'completely'

In [113]:
start_backfill='2023-07-09 00:30' # comment
end_backfill='2023-09-03 00:30'
period_index=pd.date_range(start=start_backfill,end=end_backfill, freq="W-SUN")
listLogDate=[ d.strftime('%Y-%m-%d %H:%M')   for  d in  period_index   ]
print(listLogDate)
# uncomment
# Main 
# print("Collect prediction result to monitor performance model")

# multiple items
# listLogDate=[
#     '2023-06-03 00:00','2023-06-10 00:00','2023-06-17 00:00','2023-06-24 00:00',
#     '2023-07-01 00:00','2023-07-08 00:00','2023-07-15 00:00','2023-07-22 00:00','2023-07-29 00:00',
#     '2023-08-05 00:00','2023-08-12 00:00','2023-08-19 00:00','2023-08-26 00:00','2023-09-02 00:00',
#     ] 
# listLogDate=[
#      '2023-08-05 00:00','2023-08-12 00:00','2023-08-19 00:00','2023-08-26 00:00','2023-09-02 00:00'
# ]
# for  d in listLogDate:
#   print(f"*******************************Collect prediction result as of {d}*****************************************")
#   print(collect_prediction_result(d))
#   print("************************************************************************************************")

# sigle item
# collectionDate='2023-08-26 00:00' # comment    
# print(collect_prediction_result(collectionDate))


['2023-07-09 00:30', '2023-07-16 00:30', '2023-07-23 00:30', '2023-07-30 00:30', '2023-08-06 00:30', '2023-08-13 00:30', '2023-08-20 00:30', '2023-08-27 00:30', '2023-09-03 00:30']
